In [101]:
import ugradio
import numpy as np
from __future__ import division

# 1) Converting HMS/DMS Equatorial to Decimal Equatorial

## Functions

In [4]:
def HMS(hour, minute, second):
    """
    Converts RA from hour-minute-second notation to decimal notation.
    
    Arguments:
    hour: RA value of degree. 
    minute: RA value of minute.
    second: RA value of second.
    
    Returns:
    Decimal representation of the degree-minute-second declination.
    """
    minute = minute/60
    second = second/3600
    decimal = 15*(hour + minute + second)
    return decimal
  
    
def DMS(degree, arcmin, arcsec):
    """
    Converts dec from degree-minute-second notation to decimal notation.  
    
    Arguments:
    degree: dec value of degree. 
    arcmin: dec value of arcminute.
    arcsec: dec value of arcsecond.
    
    Returns:
    Decimal representation of the degree-minute-second declination.
    """
    arcmin = arcmin/60
    arcsec = arcsec/3600
    if degree < 0:
        decimal = -(np.abs(degree) + arcmin + arcsec)
    else:
        decimal = np.abs(degree) + arcmin + arcsec
    return decimal

## Examples

In [5]:
#Polaris: The North Star, dec≈90°
print 'Polaris    |',(HMS(2,31,47.06), DMS(89,15,51)),' | North Star'

#λ Piscium: Star closest to Vernal equinox, RA≈0h
print 'λ Piscium  |',(HMS(23, 42, 2.9), DMS(1,46,49.51)),' | Vernal Equinox'

#TRAPPIST-1: Location of seven temperate terrestrial planets
print 'TRAPPIST-1 |',(HMS(23,6,29.2831), DMS(-5,2,28.59)),'          | Habitable Exoplanets' 

#Messier 87: Location of the EHT black hole photograph
print 'Messier 87 |', (HMS(12,30,49.4), DMS(12,23,26)),'| Black Hole Photo'


Polaris    | (37.946083333333334, 89.26416666666667)  | North Star
λ Piscium  | (355.5120833333333, 1.7804194444444443)  | Vernal Equinox
TRAPPIST-1 | (346.6220129166667, -5.041275)           | Habitable Exoplanets
Messier 87 | (187.70583333333335, 12.390555555555554) | Black Hole Photo


# 2) Converting Between Galactic and Equatorial

## Functions

In [6]:
def gal2equ(l, b):
    """
    Converts galactic coordinates to equatorial coordinates, according to the epoch J2000. 
    
    Arguments:
    l, Galactic longitude (degrees).
    b, Galactic latitude (degrees).
    
    Returns: 
    RA, Equatorial right ascension (degrees).
    dec, Equatorial declination (degrees).
    """
    l = np.radians(l)
    b = np.radians(b) 
    x = np.array([0.,0,0])
    x[0] = np.cos(b) * np.cos(l)
    x[1] = np.cos(b) * np.sin(l)
    x[2] = np.sin(b)
    R = np.array([-0.054876, -0.873437, -0.483835, 0.494109, -0.444830, 0.746982, -0.867666, -0.198076, 0.455984])
    R = R.reshape(3, 3)
    iR = np.linalg.inv(R)
    xp = np.dot(iR, x)
    RA = np.arctan2(xp[1], xp[0])
    dec = np.arcsin(xp[2])
    RA = np.degrees(RA)
    dec = np.degrees(dec)
    return (RA, dec)


def equ2gal(RA, dec):
    """
    Converts equatorial coordinates to galactic coordinates, according to the epoch J2000. 
    
    Arguments:
    RA, Equatorial right ascension (degrees).
    dec, Equatorial declination (degrees).
    
    Returns: 
    l, Galactic longitude (degrees).
    b, Galactic latitude (degrees).
    """
    RA = np.radians(RA)
    dec = np.radians(dec) 
    x = np.array([0.,0,0])
    x[0] = np.cos(dec) * np.cos(RA)
    x[1] = np.cos(dec) * np.sin(RA)
    x[2] = np.sin(dec)
    R = np.array([-0.054876, -0.873437, -0.483835, 0.494109, -0.444830, 0.746982, -0.867666, -0.198076, 0.455984])
    R = R.reshape(3, 3)
    xp = np.dot(R, x)
    l = np.arctan2(xp[1], xp[0])
    b = np.arcsin(xp[2])
    l = np.degrees(l)
    b = np.degrees(b)
    if l < 0:
        l = l + 360
    return (l, b)

## Examples

In [7]:
#Aaron's "canonical comparison position"
print 'Aaron   |', gal2equ(120, 0),'     | Lab 2 Cassiopiea'

#Sagittarius A*: Center of Milky Way, l≈0°
print 'Sgr A*  |', equ2gal(HMS(17,45,40.046), DMS(-29,0,28)),'| Galactic Center'

#Beta Tauri: Anticenter of Milky Way, l≈180°
print 'β Tauri |', equ2gal(HMS(5,26,17.5), DMS(28,36,28)),'  | Galactic Anticenter'

#31 Comae Berenices: North galactic pole, b≈90°
print '31 Com  |', equ2gal(HMS(12,51,41.8), DMS(27,32,27)),'    | North Galactic Pole'

#NGC 288: Sourth galactic pole, b≈-90°
print 'NGC 288 |', equ2gal(HMS(0,52,46.4), DMS(-26,35,10)),'   | South Galactic Pole'


Aaron   | (6.450801282430336, 62.72574264979434)      | Lab 2 Cassiopiea
Sgr A*  | (359.94431539014397, -0.046189046517267573) | Galactic Center
β Tauri | (177.99412720673072, -3.7452395574085124)   | Galactic Anticenter
31 Com  | (115.01398209267651, 89.58265198048663)     | North Galactic Pole
NGC 288 | (151.79369199370132, -89.38087316218108)    | South Galactic Pole


# 3) Converting Between Galactic and Horizontal

## Functions

In [48]:
def gal2hor(l, b, jd=None):
    """
    Converts galactic coordinates to horizontal coordinates at Leuschner. 
    
    Arguments:
    l, Galactic longitude (degrees).
    b, Galactic latitude (degrees).
    jd, Julian date (default=now).
    
    Returns: 
    alt, Horizontal altitude (degrees)
    az, Topocentric azimuth (degrees)
    """
    equ = equ2gal(l,b)
    leo = ugradio.leo
    hor = ugradio.coord.get_altaz(equ[0], equ[1], jd=jd, lat=leo.lat, lon=leo.lon, alt=leo.alt)
    alt = hor[0]
    if alt < 0:
        alt = alt + 90
    if alt > 90:
        alt = alt - 90
    azi = hor[1]
    if azi < 0:
        azi = alt + 360
    if azi > 360:
        azi = azi - 360
    return (alt, azi)

In [50]:
gal2hor(120, 0)

(58.569603440934245, 30.266755503079064)

# 4) Danger Zone of Leuschner

In [127]:
def danger(l, b):
    """
    Determines if given galtctic coordinates are safe or out of range for Leuschner. 
    
    Arguments:
    l, Galactic longitude (degrees).
    b, Galactic latitude (degrees).
    
    Returns:
    alt, Horizontal altitude (degrees).
    azi, Horizontal azimuth (degrees).
    """
    min_alt, max_alt = 15, 85
    min_azi, max_azi = 5, 350
    hor = gal2hor(l, b)
    alt, azi = hor[0], hor[1]
    print '(alt, azi) =',hor
    if alt < min_alt or alt > max_alt:
        print 'Coord Lord says: ALT OUT OF RANGE. DO NOT ATTEMPT POINTING.'
    if azi < min_azi or azi > max_azi:
        print 'Coord Lord says: AZI OUR OF RANGE. DO NOT ATTEMPT POINTING.'
    if alt > min_alt and alt < max_alt and azi > min_azi and azi < max_azi:
        print 'Coord Lord says: SAFE. GO FORTH AND POINT.'

In [131]:
danger(180, 0)

(alt, azi) = (9.686137767147855, 349.384333153513)
Coord Lord says: ALT OUT OF RANGE. DO NOT ATTEMPT POINTING.
